In [ ]:
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.chdir("../input/smoke-detection-dataset")
os.listdir()

In [ ]:
# patoolib.extract_archive('smoke_detection_iot.csv.zip')
os.listdir()

In [ ]:
data= pd.read_csv('smoke_detection_iot.csv')
data.head()


In [ ]:
data.columns

In [ ]:
data.drop(['Unnamed: 0', 'UTC','CNT'],axis = 1, inplace = True)
data.columns

In [ ]:
print(f"Shape of data {data.shape}")
print(f"Info of data {data.info()}")

display(data.describe())


In [ ]:
data.isnull().sum()

In [ ]:
data.skew()

In [ ]:
# vizualization data
sns.histplot(data =data,x = 'Raw H2' , hue = 'Fire Alarm',kde = True,bins = 100)


In [ ]:
sns.histplot(data =data,x = 'Temperature[C]' , hue = 'Fire Alarm',kde = True,bins = 100)


In [ ]:
sns.histplot(data =data,x = 'Pressure[hPa]' , hue = 'Fire Alarm',kde = True,bins = 100)

In [ ]:
# Split data
X = data.drop(['Fire Alarm'],axis = 1)
y = data['Fire Alarm']
cols = X.columns

In [ ]:
# Normalize data
from sklearn.preprocessing import StandardScaler
stdScaler = StandardScaler()
stdScaler.fit(X)
X = stdScaler.transform(X)

In [ ]:
X = pd.DataFrame(X,columns= cols)

In [ ]:
X.head()

In [ ]:
# Check skew
skew_limit = 0.5
skew_cols = X.skew()
skew_cols = skew_cols.sort_values(ascending = False)
skew_df = skew_cols.to_frame()
skew_df = skew_df.rename(columns = {0 : 'Skew'})
skew_cols = skew_df.query(f'abs(Skew) > {skew_limit}')
display(skew_cols)

In [ ]:
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

def NormalizeSkewedFeatures(data_modelling):
  for col in skew_cols.index:
    if(col != "SalePrice"):
      try:
        data_modelling[col] = boxcox1p(data_modelling[col], boxcox_normmax((data_modelling[col] + 1)))
      except:
        print(f"colum {col} can not apply BoxCox")
        continue
  return data_modelling

In [ ]:
X_Train = NormalizeSkewedFeatures(X.copy())

In [ ]:
X_Train['Temperature[C]'] = X['Temperature[C]']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(X, y)

In [ ]:
y_predict = clf.predict(X)
data['Fire Alarm Predict'] = y_predict
data.head(10)

In [ ]:
print(f"Accurancy of model {clf.score(X,y)}")